# Data Processing Notebook

In this notebook we showcase data processing steps done unto data upon retrieval from Kaggle.

### Download and Import Data (48.1s)

In [ ]:
import os
import shutil
import kagglehub

# Download dataset from kagglehub
path = kagglehub.dataset_download("tunguz/big-five-personality-test")
print(path)
print("Dataset path:", path)

# Specify target directory for data
target_folder = 'data/'
os.makedirs(target_folder, exist_ok=True)

# Check the contents of the dataset path
print("Files in the dataset:", os.listdir(path))

for file_name in os.listdir(path):
    source = os.path.join(path, file_name)
    destination = os.path.join(target_folder, file_name)
    
    # Check if the file exists
    if os.path.exists(source):
        shutil.move(source, destination)
        print(f"Moved {file_name} to {target_folder}")
    else:
        print(f"File does not exist: {source}")

print("Dataset moved to:", target_folder)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
import shutil
import os

/Users/juliann/opt/anaconda3/envs/project_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Clean into desired format (~40s)

In [4]:
data = pd.read_csv("../data/IPIP-FFM-data-8Nov2018/data-final.csv")
rows = []
cols = data.columns[0].split('\t')
for index in range(len(data)):
    value = data.iloc[index, 0]
    row = (value.split('\t'))
    rows.append(row)
data = pd.DataFrame(rows, columns=cols)

In [16]:
print(len(data))

696845


### Normalize Data Types (~90s)

In [5]:
non_numeric_cols = ['dateload', 'country']
numeric_cols = [col for col in data.columns if col not in non_numeric_cols]

# Convert only the selected columns to numeric
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

### Filter for only 1 user per entry (more accurate)

In [6]:
# Filter the data where 'ipc' column is equal to 1
data = data[data['IPC'] == 1]

### Set up trait score columns

In [7]:
import pandas as pd
import numpy as np

# Define score column names
score_cols = ["O score", "C score", "E score", "A score", "N score"]

# Efficiently add all new columns at once with zero values
data = pd.concat([data, pd.DataFrame(0, index=data.index, columns=score_cols)], axis=1)


### Score traits based on scoring rubic (scource: ipip.ori.org)

Provided scoring rubic is applied to dataset, using matrix multiplcation to calcualte trait scorse for teach test taker as weighted sums.

In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load the scoring sheet
scoring = pd.read_excel('../scoring/scoring.xlsx')

# Pivot the scoring table for fast lookups (wide format: ids as index, traits as columns)
scoring_pivot = scoring.pivot(index='id', columns='value', values='direction').fillna(0)

# Keep only relevant columns from data
scoring_subset = data[scoring_pivot.index]

# Initialize the new score columns to 0
for col in score_cols:
    data[col] = 0

# Apply tqdm for tracking progress
for col in tqdm(score_cols, desc="Calculating scores for traits"):
    trait = col[0]  # Extract the trait letter
    if trait in scoring_pivot.columns:
        tqdm_desc = f"Processing {trait}"
        
        # Matrix multiplication (dot product) for fast score calculation
        data[col] = tqdm(pd.Series(np.dot(scoring_subset.to_numpy(), 
                                                  scoring_pivot[trait].to_numpy()), 
                                           index=data.index), 
                                 desc=tqdm_desc, leave=False)

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err,O score,C score,E score,A score,N score
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,6,1,GB,51.5448,0.1991,27.0,8.0,16.0,15.0,-12.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,11,1,MY,3.1698,101.7060,17.0,13.0,-10.0,20.0,-13.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,7,1,GB,54.9119,-1.3833,23.0,10.0,-5.0,18.0,-14.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,7,1,GB,51.7500,-1.2500,21.0,1.0,-4.0,14.0,-17.0
5,3.0,3.0,4.0,2.0,4.0,2.0,2.0,3.0,3.0,4.0,...,3,1,SE,59.3333,18.0500,26.0,13.0,2.0,14.0,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015334,4.0,1.0,4.0,2.0,4.0,3.0,3.0,2.0,3.0,2.0,...,16,1,GB,52.4189,-1.5054,18.0,10.0,8.0,16.0,-16.0
1015335,5.0,1.0,5.0,2.0,4.0,2.0,3.0,1.0,5.0,1.0,...,9,1,TR,41.0186,28.9647,22.0,3.0,15.0,15.0,-34.0
1015337,4.0,3.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,3.0,...,7,1,US,38.0000,-97.0000,27.0,4.0,2.0,15.0,-32.0
1015339,2.0,4.0,3.0,4.0,2.0,2.0,1.0,4.0,2.0,4.0,...,8,1,US,34.1067,-117.8067,23.0,14.0,-8.0,13.0,-23.0


### Save full dataset and trait scores to local CSV files

In [18]:
data.to_csv('../data/cleaned_data_v2.csv')
#data.iloc[:, -5:].to_csv("../data/trait_scores.csv", index=False)